# #2 Preprocesamiento de Variables

Este notebook está dedicado al preprocesamiento de variables en distintos conjuntos de datos. El preprocesamiento es una fase esencial en el ciclo de vida de un proyecto de análisis de datos, ya que prepara el terreno para análisis posteriores y el entrenamiento de modelos predictivos.

## Objetivos del Notebook

Los principales objetivos de este notebook son:

- Limpiar y transformar las variables para facilitar su análisis y modelado.
- Estandarizar el formato de los datos para su integración y comparación.
- Enriquecer el dataset con nuevas variables calculadas o derivadas para obtener mayor valor analítico.
- Reducir la dimensionalidad de los datos para simplificar el entrenamiento de los modelos y mejorar su rendimiento.

## Contenido del Notebook

El contenido de este notebook se divide en las siguientes secciones:

1. **Limpieza de Datos**: En esta sección, utilizaremos la información obtenida a partir del EDA (ver notebook #1) para corregir los principales errores y anomalías con las que cuenta el dataframe.

2. **Modificación y Generación de Nuevas Variables:** Modificaremos algunas de las variables ya existentes y crearemos algunas nuevas que puedan ser más informativas y útiles para los modelos predictivos.

3. **Transformación de Variables:** Aplicaremos técnicas de transformación de variables, como la normalización, la estandarización y la conversión de variables categóricas en numéricas, para preparar el dataframe para la siguiente etapa del proyecto.

4. **Exportación del Modelo Modificado:** Finalmente, exportaremos el dataframe obtenido a partir de todas las correcciones y modificaciones realizadas al conjunto de datos original, para utilizarlo durante el entrenamiento y evaluación del modelo predictivo.

Cada sección incluirá ejemplos prácticos y código para llevar a cabo cada una de las tareas de preprocesamiento mencionadas.

## Autores
- **Miguel Guerrero Ruiz**
- **Bastián Sepúlveda Silva**

## Librerías utilizadas

En este notebook se realiza la importación de varias librerías necesarias para el preprocesamiento de datos y su visualización. A continuación, se detallan todas las librerías utilizadas:

- `pandas`: Proporciona estructuras de datos y herramientas para la manipulación y análisis efectivo de los mismos.
- `numpy`: Ofrece soporte para arrays y matrices grandes y multidimensionales, junto con una colección de funciones matemáticas para operar en estos arrays.
- `scikit-learn`: Imprescindible para aplicar algoritmos de preprocesamiento, modelado y reducción de dimensionalidad.


## Instalacion de paquetes necesarios

In [ ]:
# Se instala category_encoders ya que es utilizado durante la codificación de variables categóricas
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
# Se importan las librerías a utilizar
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from google.colab import drive

# Se crea una instancia de Google Drive desde la cual se cargarán los datos
#drive.mount('/content/drive/') # descomentar para cargar los datos desde Google Drive

Mounted at /content/drive/


## Lectura de datos

In [ ]:
# Se lee el archivo en formato parquet
file_path = './pronostico_estadia.parquet'
df= pd.read_parquet(file_path)

## 1. Limpieza de Datos
En esta sección, utilizaremos la información obtenida a partir del EDA (ver notebook #1) para corregir los principales errores y anomalías con las que cuenta el dataframe. Para ello, se realizarán los siguientes pasos:

- Se buscará la cantidad de datos duplicados, eliminando las entradas idénticas y dejando solo una.
- Se corregirán los errores y anomalías detectadas durante el EDA, entre ellas: valores faltantes en todas las columnas, pero con una mayor cantidad en la columna `Tiempo Gestación`; valores atípicos en la columna `Sexo (Desc)`.

### Búsqueda y corrección de valores duplicados

In [ ]:
# Para verificar duplicados
# Se busca el número de duplicados
n_duplicates = df.duplicated().sum()
print(f"Hay {n_duplicates} duplicados.")

# Se extraen los nombres de las columnas que coincidan
columns_to_consider = df.columns

# Si existen duplicados, se eliminan dichas columnas
df = df.drop_duplicates(subset=columns_to_consider)

Hay 681 duplicados.


In [ ]:
# Se muestra la cantidad de duplicados lugo de ser eliminados
n_duplicates = df.duplicated().sum()
print(f"Hay {n_duplicates} duplicados.")

Hay 0 duplicados.


### Corrección de errores y anomalías detectadas durante el EDA

In [ ]:
# Se normaliza la variable 'Tiempo Gestación', llenando con 0 los valores faltantes.
df['Tiempo Gestación'] = df['Tiempo Gestación'].apply(lambda x: x if pd.notna(x) and x > 0 else 0)

**Observaciones:** La decisión de rellenar con 0 (ceros) se tomó en base a lo observado durante el EDA, en el cual se descubrió que, a partir del año 2020, los tiempos de gestación inexistentes pasaron de ser representados por ceros a valores en blanco (NaN).

In [ ]:
# Se elimina la fila que cuenta con datos faltantes en todas sus columnas
df.dropna(axis=0, how='all', inplace=True)

**Observaciones:** Durante el EDA, se notó que existía un valor faltante para cada columna del dataframe, al inspeccionar el archivo directamente, se descubrió que se trataba de una fila que contaba sin ningún dato. Simplemente se procedió a eliminarla.

In [ ]:
# Se eliminan las filas que cuentan con sexo "Intersexo" o "Desconocido", debido a la poca relevancia que tienen para el caso
df = df.loc[df['Sexo (Desc)'].isin(['Hombre', 'Mujer'])]
num_unique_values_sexo = df['Sexo (Desc)'].nunique()
print(f'Número de categorías únicas para Sexo: {num_unique_values_sexo}')

Número de categorías únicas para Sexo: 2


**Observaciones:** Durante el EDA, se observaron valores atípicos para la variable 'Sexo (Desc)', los cuales no aportaban mucho para el modelo, debido a la baja cantidad de su frecuencia (existía una única fila para cada uno). Se tomó la decisión de eliminar las filas correspondientes a estos valores.

## 2. Modificación y Generación de Nuevas Variables

En esta sección, modificaremos algunas de las variables ya existentes y crearemos algunas nuevas que puedan ser más informativas y útiles para los modelos predictivos. Para ello, se realizarán los siguientes pasos:

- Se segmentarán las edades en grupos para reducir la complejidad en la relación no lineal entre la instancia hospitalaria (variable objetivo) y una edad exacta.
- Se extraerá el CIE de cada diagnósito y procedimiento para ser utilizado de ahora en adelante.
- Se aplicará una transformación logarítmica a la variable objetivo con el propósito de reducir la simetría normalizando la distribución de los datos y, a su vez, disminuir el impacto de los *outliers*.
- Se crearán nuevas características con el objetivo de generar información condensada en ellas y que esta información pueda revelar patrones y relaciones que no son inmediatamente evidentes en los datos brutos.

### Segmentación de edades

In [ ]:
# Se realiza la segmentación de edades
def age_group(age):
    if age <= 2:
        return "Infantes"
    elif age <= 12:
        return "Niños"
    elif age <= 18:
        return "Adolescentes"
    elif age <= 30:
        return "Jóvenes adultos"
    elif age <= 60:
        return "Adultos"
    else:
        return "Adultos mayores"
df["Edad Agrupada"] = df["Edad en años"].apply(age_group)
df.drop(columns=["Edad en años"], inplace=True)

**Observaciones:** Para efectos de la segmentación, se escogieron rangos de edad de acuerdo con las distintas etapas significativas del desarrollo humano.

### Extracción de CIE (Código Internacional de Enfermedades)

In [ ]:
# Se extraen los CIE de las columnas de diagnósticos y procedimientos
def extract_code(text):
    return str(text).split(' - ')[0]

diag_cols = [col for col in df.columns if "Diag" in col]
proced_cols = [col for col in df.columns if "Proced" in col]

for col in diag_cols + proced_cols:
    df[col] = df[col].apply(extract_code)

for col in diag_cols + proced_cols:
    df[col] = df[col].apply(lambda x: ''.join(filter(str.isdigit, str(x))))

**Observaciones:** En base a la información obtenida en el EDA (ver notebook #1), se optó por utilizar el CIE (Código Internacional de Enfermedades), el cual se encuentra escrito en cada diagnóstico y procedimiento, en lugar del texto completo contenido en cada uno de ellos.

### Transformación logarítmica de la variable objetivo

In [ ]:
# Se aplica transformación logarítmica a "Estancia del Episodio"
df["Log_Estancia"] = df["Estancia del Episodio"].apply(lambda x: np.log1p(x))
df.drop(columns=['Estancia del Episodio'], inplace=True)

**Observaciones:** Como se visualizó en el EDA (ver notebook #1), la variable `Estancia del Episodio` tenía un claro sesgo en un rango entre [0-5], a raíz de esto, se optó por realizar una transformación logarítmica para mejorar la calidad de los resultados.

### Creación de nuevas características

In [ ]:
# Se crea una nueva característica con la suma total de diagnósticos y procedimientos
df["Total_Diag"] = df[[col for col in diag_cols]].nunique(axis=1)
df["Total_Proced"] = df[[col for col in proced_cols]].nunique(axis=1)

# Se crea una nueva característica con la interacción entre el diagnóstico principal y el procedimiento principal
df["Diag01_Proced01"] = df["Diag 01 Principal (cod+des)"] + "_" + df["Proced 01 Principal (cod+des)"]

# Se crea una nueva característica con la cantidad de diagnósticos secundarios
# Excluímos el diagnóstico principal y contamos los diagnósticos únicos
df["Num_Diag_Secundarios"] = df[[col for col in diag_cols if col != "Diag 01 Principal (cod+des)"]].nunique(axis=1)

# Se crea una nueva característica con la cantidad de procedimientos secundarios
# Excluímos el procedimiento principal y contamos los procedimientos únicos
df["Num_Proced_Secundarios"] = df[[col for col in proced_cols if col != "Proced 01 Principal (cod+des)"]].nunique(axis=1)

**Observaciones:** Se tomó la decisión de crear principalmente características relacionadas a los diagnósticos y procedimientos, debido a que son las variables en las cuales sería de mayor utilidad encontrar patrones y relaciones.

## 3. Transformación de Variables

En esta sección, aplicaremos técnicas de transformación de variables, como la normalización, la estandarización y la conversión de variables categóricas en numéricas, para preparar el dataframe para la siguiente etapa del proyecto. Para ello, se realizarán los siguientes pasos:

- Se codificarán las variables categóricas en numéricas según su cardinalidad.
- Se revisará que la codificación se realizara con éxito.
- Se estandarizarán las columnas numéricas del dataframe, incluyendo las previamente codificadas.

### Codificación de las variables categóricas

Se utilizan dos métodos para codificar dependiendo de la cardinalidad de las variables. `Target Encoding` para alta cardinalidad y `One Hot Encoding` para baja cardinalidad.

In [ ]:
# Se revisan los tipos de datos de cada variable antes de codificar
print(df.dtypes)

Sexo (Desc)                        object
Especialidad (Descripción )        object
Tipo Ingreso (Descripción)         object
Servicio Ingreso (Código)          object
Tiempo Gestación                  float64
Diag 01 Principal (cod+des)        object
Diag 02 Secundario (cod+des)       object
Diag 03 Secundario (cod+des)       object
Diag 04 Secundario (cod+des)       object
Diag 05 Secundario (cod+des)       object
Diag 06 Secundario (cod+des)       object
Diag 07 Secundario (cod+des)       object
Diag 08 Secundario (cod+des)       object
Proced 01 Principal (cod+des)      object
Proced 02 Secundario (cod+des)     object
Proced 03 Secundario (cod+des)     object
Proced 04 Secundario (cod+des)     object
Proced 05 Secundario (cod+des)     object
Proced 06 Secundario (cod+des)     object
Proced 07 Secundario (cod+des)     object
Proced 08 Secundario (cod+des)     object
Proced 09 Secundario (cod+des)     object
Proced 10 Secundario (cod+des)     object
Edad Agrupada                     

In [ ]:
# 1. Target Encoding
# Lista de columnas para Target Encoding
target_encoding_cols_updated = [
    "Especialidad (Descripción )",
    "Servicio Ingreso (Código)",
    "Diag 01 Principal (cod+des)",
    "Diag 02 Secundario (cod+des)",
    "Diag 03 Secundario (cod+des)",
    "Diag 04 Secundario (cod+des)",
    "Diag 05 Secundario (cod+des)",
    "Diag 06 Secundario (cod+des)",
    "Diag 07 Secundario (cod+des)",
    "Diag 08 Secundario (cod+des)",
    "Proced 01 Principal (cod+des)",
    "Proced 02 Secundario (cod+des)",
    "Proced 03 Secundario (cod+des)",
    "Proced 04 Secundario (cod+des)",
    "Proced 05 Secundario (cod+des)",
    "Proced 06 Secundario (cod+des)",
    "Proced 07 Secundario (cod+des)",
    "Proced 08 Secundario (cod+des)",
    "Proced 09 Secundario (cod+des)",
    "Proced 10 Secundario (cod+des)",
    "Diag01_Proced01"
]

# Se inicializa el codificador
target_encoder_updated = ce.TargetEncoder(cols=target_encoding_cols_updated)

# Se ajustan y transforman las columnas con Target Encoding
df_encoded = target_encoder_updated.fit_transform(df, df['Log_Estancia'])

In [ ]:
# 2. One Hot Encoding
# Columnas para One Hot Encoding
one_hot_cols_updated = ['Sexo (Desc)', 'Tipo Ingreso (Descripción)', 'Edad Agrupada']

# Se aplica One Hot Encoding en las columnas correspondientes
df_encoded = pd.get_dummies(df_encoded, columns=one_hot_cols_updated, drop_first=True)

### Revisión de la codificación de los datos

In [ ]:
# Se revisan los tipos de datos nuevamente para verificar los cambios
print(df_encoded.dtypes)

Especialidad (Descripción )              float64
Servicio Ingreso (Código)                float64
Tiempo Gestación                         float64
Diag 01 Principal (cod+des)              float64
Diag 02 Secundario (cod+des)             float64
Diag 03 Secundario (cod+des)             float64
Diag 04 Secundario (cod+des)             float64
Diag 05 Secundario (cod+des)             float64
Diag 06 Secundario (cod+des)             float64
Diag 07 Secundario (cod+des)             float64
Diag 08 Secundario (cod+des)             float64
Proced 01 Principal (cod+des)            float64
Proced 02 Secundario (cod+des)           float64
Proced 03 Secundario (cod+des)           float64
Proced 04 Secundario (cod+des)           float64
Proced 05 Secundario (cod+des)           float64
Proced 06 Secundario (cod+des)           float64
Proced 07 Secundario (cod+des)           float64
Proced 08 Secundario (cod+des)           float64
Proced 09 Secundario (cod+des)           float64
Proced 10 Secundario

In [ ]:
# Se verifica la ausencia de valores nulos
print(df_encoded.isnull().sum())

Especialidad (Descripción )              0
Servicio Ingreso (Código)                0
Tiempo Gestación                         0
Diag 01 Principal (cod+des)              0
Diag 02 Secundario (cod+des)             0
Diag 03 Secundario (cod+des)             0
Diag 04 Secundario (cod+des)             0
Diag 05 Secundario (cod+des)             0
Diag 06 Secundario (cod+des)             0
Diag 07 Secundario (cod+des)             0
Diag 08 Secundario (cod+des)             0
Proced 01 Principal (cod+des)            0
Proced 02 Secundario (cod+des)           0
Proced 03 Secundario (cod+des)           0
Proced 04 Secundario (cod+des)           0
Proced 05 Secundario (cod+des)           0
Proced 06 Secundario (cod+des)           0
Proced 07 Secundario (cod+des)           0
Proced 08 Secundario (cod+des)           0
Proced 09 Secundario (cod+des)           0
Proced 10 Secundario (cod+des)           0
Log_Estancia                             0
Total_Diag                               0
Total_Proce

In [ ]:
# Se verifica la cantidad de columnas
print(f"Antes de codificar: {len(df.columns)} columnas")
print(f"Después de codificar: {len(df_encoded.columns)} columnas")

Antes de codificar: 30 columnas
Después de codificar: 36 columnas


**Observaciones:** El número de columnas es mayor al finalizar el proceso debido a la codificación realizada.

### Estandarización de variables

Se estandarizan las columnas numéricas para tener una media de 0 y una desviación estándar de 1.

In [ ]:
# Se inicializa el estandarizador
scaler = StandardScaler()

# Se seleccionan las columnas numéricas
numeric_features = df_encoded.select_dtypes(include=['float64', 'int64']).columns

# Se estandarizan las características numéricas
df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])


## 4. Exportación del Dataframe Modificado

En esta sección, exportaremos el dataframe obtenido a partir de todas las correcciones y modificaciones realizadas al conjunto de datos original, para utilizarlo durante el entrenamiento y evaluación del modelo predictivo. Este dataframe incluye todos los cambios realizados durante este notebook.

In [ ]:
# Se crea un archivo parquet con el dataframe como se dejó hasta el momento
ruta_parquet = './dataset_pronostico_estadia.parquet'
df_encoded.to_parquet(ruta_parquet, index=False)